# Introduction 

This notebook will generate a common employment data with following columns :
1. ID : Employe ID
2. DOB : Date of Birth
3. DOH : Date of Hire
4. Salary

The aim of this notebook is that Consulting Actuary Company or any other company that work with employee data can have an open employee data that they can generate on their own for training or education purposes.

# Data Generator

To minimize the error of data, the range of date of the data will be between 1 to 28 (instead of 1 to 31 like normal calendar). 

In [1]:
# Packages
import pandas as pd # DataFrame
import numpy as np # Vector or Matrices
import datetime # Date time

In [2]:
def generate_date(year_1 = 1980, year_2 = 2000):
    return datetime.datetime(year_1, 1, 1) + datetime.timedelta(days= np.random.randint((datetime.datetime(year_2, 12,31) - datetime.datetime(year_1, 1,1)).days))  


In [3]:
emp_counts = 10

# Creating DataFrame
data = pd.DataFrame(data = {"id" : np.random.randint(0, emp_counts*10, emp_counts), 
                            'dob' : [generate_date() for i in range(emp_counts)],
                            'doh' : [generate_date(year_1 = 2008, year_2 = 2023) for i in range(emp_counts)],
                            'salary' : np.round(np.random.randn(emp_counts)**2*1e4,2)})
data['dob'] = pd.to_datetime(data['dob'])
data['doh'] = pd.to_datetime(data['doh'])
data

,id,dob,doh,salary
0,5,1992-06-29,2018-04-28,358.08
1,97,1983-03-14,2009-11-06,23624.10
2,64,1993-07-17,2012-05-31,6859.72
3,8,1987-10-12,2023-08-25,1170.39
4,6,1990-02-10,2012-10-05,45433.45
5,46,1989-01-16,2017-07-26,3818.31
6,73,1997-09-16,2013-01-01,145.50
7,39,1998-02-10,2021-10-17,12172.49
8,87,1982-05-07,2013-09-16,36906.32
9,50,1990-02-19,2014-07-02,9742.10


In [4]:
# To save in Excel Format
#data.to_excel(f'data/employee-data-with-{emp_counts}-data.xlsx')

# Actuarial Assumption 
## Mortality and Morbidity Distribution

We will make a mortality Table from Makeham Distribution with parameters as follow 
$A = 0.00022, B = 2.7 × 10^{−6}, c = 1.124$. In general, the Survival Function of Makeham Distribution is :
\begin{equation}
S_X(t) = \exp(-A)\exp\left(\left(-\frac{B}{\ln{c}}c^x\left(c^t-1\right)\right)\right)
\end{equation}

In [5]:
def p(t,x, A = 0.00022, B = 2.7e-06, c = 1.124) :
    return np.exp(-A)*np.exp((-B/np.log(c)*c**x*(c**t-1)))

We also assume that Morbidity is 1% of the Mortality Rate

## Resignation Rate
Since assumption on resignation rate can be vary across company, we will use a simple assumption that is the rate is 10% at the age of 22 and decrease linearly to 1% at the age of retirement - 1.

# Economic Assumption 
Since the yield curve data hasn't been modeled in this repository, we assume a single discount rate with 5% p.a. and salary increase 10% p.a.
We also assume that the severance, service and seperation pay as follow :
| Yos | Sev | Svc |
|:---:|:---:|:---:|
|  0  |  1  |  0  |
|  1  |  2  |  0  |
|  2  |  3  |  0  |
|  3  |  4  |  2  |
|  4  |  5  |  2  |
|  5  |  6  |  2  |
|  6  |  7  |  3  |
|  7  |  8  |  3  |
|  8  |  9  |  3  |
|  9  |  9  |  4  |
|  10 |  9  |  4  |
|  11 |  9  |  4  |
|  12 |  9  |  5  |
|  13 |  9  |  5  |
|  14 |  9  |  5  |
|  15 |  9  |  6  |
|  16 |  9  |  6  |
|  17 |  9  |  6  |
|  18 |  9  |  7  |
|  19 |  9  |  7  |
|  20 |  9  |  7  |
|  21 |  9  |  8  |
|  22 |  9  |  8  |
|  23 |  9  |  8  |
|  24 |  9  |  10 |
|  25 |  9  |  10 |
|  26 |  9  |  10 |
|  27 |  9  |  10 |
|  28 |  9  |  10 |
|  29 |  9  |  10 |
|  30 |  9  |  10 |  

In [6]:
sev_svc = pd.DataFrame({'severance': [min(i+1,9) for i in range(60)],
                        'service' : [0,0,0,2,2,2,3,3,3,
                                     4,4,4,5,5,5,6,6,6,
                                     7,7,7,8,8,8,10,10,10,10,10,10,10,
                                     10,10,10,10,10,10,10,10,10,10,10,10,10,10,
                                     10,10,10,10,10,10,10,10,10,10,10,10,10,10,10]})
sev_svc

,severance,service
0,1,0
1,2,0
2,3,0
3,4,2
4,5,2
5,6,2
6,7,3
7,8,3
8,9,3
9,9,4


And a benefit table as follow: 

| YoS | Pension | Death | Disability | Resign |
|:---:|:-------:|:-----:|:----------:|:------:|
|  0  |   1.75  |   2   |      2     |   0.5  |
|  1  |   3.5   |   4   |      4     |    1   |
|  2  |   5.25  |   6   |      6     |   1.5  |
|  3  |    9    |   10  |     10     |    2   |
|  4  |  10.75  |   12  |     12     |   2.5  |
|  5  |   12.5  |   14  |     14     |    3   |
|  6  |  15.25  |   17  |     17     |   3.5  |
|  7  |    17   |   19  |     19     |    4   |
|  8  |  18.75  |   21  |     21     |   4.5  |
|  9  |  19.75  |   22  |     22     |   4.5  |
|  10 |  19.75  |   22  |     22     |   4.5  |
|  11 |  19.75  |   22  |     22     |   4.5  |
|  12 |  20.75  |   23  |     23     |   4.5  |
|  13 |  20.75  |   23  |     23     |   4.5  |
|  14 |  20.75  |   23  |     23     |   4.5  |
|  15 |  21.75  |   24  |     24     |   4.5  |
|  16 |  21.75  |   24  |     24     |   4.5  |
|  17 |  21.75  |   24  |     24     |   4.5  |
|  18 |  22.75  |   25  |     25     |   4.5  |
|  19 |  22.75  |   25  |     25     |   4.5  |
|  20 |  22.75  |   25  |     25     |   4.5  |
|  21 |  23.75  |   26  |     26     |   4.5  |
|  22 |  23.75  |   26  |     26     |   4.5  |
|  23 |  23.75  |   26  |     26     |   4.5  |
|  24 |  25.75  |   28  |     28     |   4.5  |
|  25 |  25.75  |   28  |     28     |   4.5  |
|  26 |  25.75  |   28  |     28     |   4.5  |
|  27 |  25.75  |   28  |     28     |   4.5  |
|  28 |  25.75  |   28  |     28     |   4.5  |
|  29 |  25.75  |   28  |     28     |   4.5  |
|  30 |  25.75  |   28  |     28     |   4.5  |

In [7]:
ben_fac = pd.DataFrame({'retire': 1.75*sev_svc['severance']+sev_svc['service'],
                        'death': 2*sev_svc['severance']+sev_svc['service'],
                        'disable': 2*sev_svc['severance']+sev_svc['service'],
                        'resign': [1]*sev_svc.shape[0]})
ben_fac

,retire,death,disable,resign
0,1.75,2,2,1
1,3.50,4,4,1
2,5.25,6,6,1
3,9.00,10,10,1
4,10.75,12,12,1
5,12.50,14,14,1
6,15.25,17,17,1
7,17.00,19,19,1
8,18.75,21,21,1
9,19.75,22,22,1


## Pre-Processing Data
We need the information of **Age** and **Year of Service (YOS)** of each of the employees. For that we also need to define the **valuation date**. Let's assume the valuation date is 31 December 2022

In [8]:
val_date = pd.Timestamp('2022-12-31')
data['age'] = np.round((val_date- data.dob)/np.timedelta64(1, 'Y'),2)
data['yos'] = np.round((val_date- data.doh)/np.timedelta64(1, 'Y'),2)

In [9]:
data

,id,dob,doh,salary,age,yos
0,5,1992-06-29,2018-04-28,358.08,30.51,4.68
1,97,1983-03-14,2009-11-06,23624.10,39.80,13.15
2,64,1993-07-17,2012-05-31,6859.72,29.46,10.58
3,8,1987-10-12,2023-08-25,1170.39,35.22,-0.65
4,6,1990-02-10,2012-10-05,45433.45,32.89,10.24
5,46,1989-01-16,2017-07-26,3818.31,33.96,5.43
6,73,1997-09-16,2013-01-01,145.50,25.29,10.00
7,39,1998-02-10,2021-10-17,12172.49,24.89,1.20
8,87,1982-05-07,2013-09-16,36906.32,40.65,9.29
9,50,1990-02-19,2014-07-02,9742.10,32.86,8.50


# Calculating the Post Employment Benefit

In [98]:
def present_value(df = data, sal_inc = 0.03,i = 0.05, nra = 55):
    sum = [0]*df.shape[0]
    for n in range(df.shape[0]):
        inc_ = np.array([(1+sal_inc)**k for k in range(int(nra) - int(df.age[n]))])
        pv = np.array([(1+i)**(-k) for k in range(int(nra) - int(df.age[n]))])               
        ben_ = ben_fac.iloc[int(df.yos[n]):(int(df.yos[n]) + int(nra) - int(df.age[n])), 1:].T.dot(np.multiply(df.salary[n]*inc_,pv))
        sum[n] = ben_.sum() + ben_fac.iloc[int(data.yos[n] + nra - data.age[n]),0]*df.salary[n]*(1+sal_inc)**(data.yos[n] + nra - data.age[n])*(1+i)**(-data.yos[n] + nra - data.age[n])
    return sum

In [96]:
df = data.copy()
sal_inc = 0.03 
i = 0.05 
nra = 55
n = 0

inc_ = np.array([(1+sal_inc)**k for k in range(int(nra) - int(df.age[n]))])
pv = np.array([(1+i)**(-k) for k in range(int(nra) - int(df.age[n]))])               
ben_ = ben_fac.iloc[int(df.yos[n]):(int(df.yos[n]) + int(nra) - int(df.age[n])), 1:].T.dot(np.multiply(df.salary[n]*inc_,pv))
sum = ben_.sum() + ben_fac.iloc[int(data.yos[n] + nra - data.age[n]),0]*df.salary[n]*(1+sal_inc)**(data.yos[n] + nra - data.age[n])*(1+i)**(-data.yos[n] + nra - data.age[n])

In [99]:
present_value()

[391604.26014298486,
 18658251.005368162,
 8469936.536217652,
 827539.779727969,
 49572352.12550166,
 3713584.386754961,
 208295.5810482619,
 15667226.924308889,
 25936198.771869246,
 10457067.28748081]